In [55]:
from scipy.stat import norm


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels import stats
import scipy, glob, os, openpyxl, time, socket, datetime
from platform import python_version
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.linear_model import Lasso
from linearmodels.panel import PanelOLS
from scipy.stats.mstats import winsorize
from statistics import quantiles

print("【日期時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【工作目錄】{}".format(os.getcwd()))
print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.gethostname())))
print("【Python】{}".format(python_version()))

%autosave 120

【日期時間】2023/03/22 17:16:07
【工作目錄】c:\Users\user\Desktop\ICO_face
【主機名稱】DESKTOP-AHINHHC (10.201.25.143)
【Python】3.9.5


Autosaving every 120 seconds


##### 匯入檔案

In [25]:
fname = r'C:\Users\user\Desktop\ICO_face\trading_ICOname2.xlsx'
file = pd.read_excel(fname, sheet_name = 'logistic')
df = file.dropna(subset = ['FWHR'], how = 'all')
df2 = df[df['Raised'] != 0]
df2 = df2[df2['Total supply'] != 0]
df2 = df2[df2['Followers'] != 0]
df2 = df2[df2['Teamsize'] != 0]
df2 = df2[df2['degree'] != 4]
df2 = df2[df2['department'] != 3]
df2 = df2[df2['work'] != 3]
# df2 = df2.dropna(axis = 0, how = 'all', subset = ['price_season5', 'price_season6', 'price_season7', 'price_season8'])
df2 = df2.dropna(how = 'any', subset = ['Date', 'duration', 'google_trend2', 'country'])
# df2 = df2[df2['google_trend2'] != 1]
df2.reset_index(drop = True, inplace = True)
# df2 = df2.set_index(['country', 'year']).sort_index(level = 'year')
# df2.to_csv(r'C:\Users\user\Desktop\trading_ICOname3.csv', sep = ',', encoding = 'utf-8')
# df2 = df2.set_index(['country', 'Date']).sort_index(level = 'Date')
# df2

##### 敘述統計量

In [100]:
aa = df2[['price_season1', 'price_season2', 'price_season3', 'price_season4', 'price_season5', 'price_season6', 'price_season7', 'price_season8']].dropna(axis = 0, how = 'all', subset = ['price_season8']) # .describe()

In [23]:
df2['Raised'].describe()

count    2.000000e+02
mean     1.878627e+07
std      2.344190e+07
min      1.141450e+05
25%      6.295294e+06
50%      1.500000e+07
75%      2.318750e+07
max      2.570000e+08
Name: Raised, dtype: float64

In [102]:
# aa = df2[['price_season1', 'price_season2', 'price_season3', 'price_season4', 'price_season5', 'price_season6', 'price_season7', 'price_season8']]
lst = []
# aa = df2['price_season7'].dropna()
for i in range(10, 100, 10):
    lst.append(np.percentile(aa, i, axis = 0))
pd.DataFrame(lst,
            columns = ['price_season1', 'price_season2', 'price_season3', 'price_season4', 'price_season5', 'price_season6', 'price_season7', 'price_season8'],
            index=['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%'])

,price_season1,price_season2,price_season3,price_season4,price_season5,price_season6,price_season7,price_season8
10%,0.004601,0.002350,0.001027,0.001193,0.001070,0.000818,0.000430,0.000386
20%,0.012760,0.006474,0.003653,0.003673,0.002307,0.001955,0.001806,0.001509
30%,0.024657,0.014116,0.010100,0.006984,0.005195,0.004402,0.003249,0.003154
40%,0.051069,0.029808,0.020006,0.016934,0.013166,0.009018,0.006672,0.006564
50%,0.116370,0.073722,0.043202,0.030980,0.021959,0.018776,0.015339,0.012435
60%,0.192341,0.146024,0.084810,0.050262,0.038773,0.032718,0.023949,0.023608
70%,0.334317,0.227992,0.154891,0.102664,0.088053,0.059397,0.052483,0.053275
80%,0.560677,0.558578,0.360599,0.235970,0.148445,0.114322,0.116014,0.119659
90%,1.587041,1.070775,0.957444,0.606988,0.427699,0.381704,0.417325,0.396014


##### 資料分析

In [26]:
df2['Non_white'] = df2['Non_white'].astype(np.int64)
df2['Female'] = df2['Female'].astype(np.int64)
df2['KYC'] = df2['KYC'].astype(np.int64)
# df2['Date'] = df2['Date'].astype(int)

##### 募資金額、google_trend分組

In [ ]:
for i in range(0, len(df2['google_trend'])):
    if df2['price_season5'][i] > 0:
        df2['trading'][i] = 1
    else:
        df2['trading'][i] = 0

In [27]:
aa = df2['Raised']
bb = np.percentile(aa, 33, axis = 0)

In [27]:
# df2 = df2[df2['Raised'] <= df2['Raised'].median()]
# df2 = df2[df2['google_trend2'] == 0]
# df2.reset_index(drop=True, inplace=True)
aa = df2['Raised'].median()
for i in range(0, len(df2)):
    if df2['Raised'][i] <= aa:
        df2['Raised'][i] = 1   # 募資金額小於中位數dummy設為1
    else:
        df2['Raised'][i] = 0
df2['Raised'].value_counts()

C:\Users\user\AppData\Local\Temp/ipykernel_13264/1386325461.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Raised'][i] = 1   # 募資金額小於中位數dummy設為1
C:\Users\user\AppData\Local\Temp/ipykernel_13264/1386325461.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Raised'][i] = 0


1.0    101
0.0     89
Name: Raised, dtype: int64

In [ ]:
df2 = df2[df2['Raised'] == 1]
df2.reset_index(drop = True, inplace = True)
df2

In [36]:
df2['Raised']

0      1.0
1      0.0
2      0.0
3      1.0
4      1.0
      ... 
185    1.0
186    1.0
187    1.0
188    1.0
189    1.0
Name: Raised, Length: 190, dtype: float64

##### 標準化

In [28]:
#建立StandardScaler物件
zscore = preprocessing.StandardScaler()
# 將季市值標準化
# data_zs1 = zscore.fit_transform(df2['season1'].values.reshape(-1, 1))
# data_zs2 = zscore.fit_transform(df2['season2'].values.reshape(-1, 1))
# data_zs3 = zscore.fit_transform(df2['season3'].values.reshape(-1, 1))
# data_zs4 = zscore.fit_transform(df2['season4'].values.reshape(-1, 1))

# 募資金額標準化
data_zs = zscore.fit_transform(df2['Raised'].values.reshape(-1, 1))

# 貨幣供給量標準化
data_zs5 = zscore.fit_transform(df2['Total supply'].values.reshape(-1, 1))

# my_list = map(lambda x: x[0], data_zs2)
# tt = pd.Series(my_list)

# 價格標準化
data_zs6 = zscore.fit_transform(df2['price_season1'].values.reshape(-1, 1))
data_zs7 = zscore.fit_transform(df2['price_season2'].values.reshape(-1, 1))
data_zs8 = zscore.fit_transform(df2['price_season3'].values.reshape(-1, 1))
data_zs9 = zscore.fit_transform(df2['price_season4'].values.reshape(-1, 1))
data_zs10 = zscore.fit_transform(df2['price_season5'].values.reshape(-1, 1))
data_zs11 = zscore.fit_transform(df2['price_season6'].values.reshape(-1, 1))
data_zs12 = zscore.fit_transform(df2['price_season7'].values.reshape(-1, 1))
data_zs13 = zscore.fit_transform(df2['price_season8'].values.reshape(-1, 1))
# volume_zs1 = zscore.fit_transform(df2['volume_season1'].values.reshape(-1, 1))
# volume_zs2 = zscore.fit_transform(df2['volume_season2'].values.reshape(-1, 1))
# volume_zs3 = zscore.fit_transform(df2['volume_season3'].values.reshape(-1, 1))
# volume_zs4 = zscore.fit_transform(df2['volume_season4'].values.reshape(-1, 1))
# volume_zs5 = zscore.fit_transform(df2['volume_season5'].values.reshape(-1, 1))
# volume_zs6 = zscore.fit_transform(df2['volume_season6'].values.reshape(-1, 1))
# volume_zs7 = zscore.fit_transform(df2['volume_season7'].values.reshape(-1, 1))
# volume_zs8 = zscore.fit_transform(df2['volume_season8'].values.reshape(-1, 1))

print('OK')

OK


In [29]:
# z_season1 = pd.DataFrame(data_zs1, columns = ['z_season1'])
# z_season2 = pd.DataFrame(data_zs2, columns = ['z_season2'])
# z_season3 = pd.DataFrame(data_zs3, columns = ['z_season3'])
# z_season4 = pd.DataFrame(data_zs4, columns = ['z_season4'])
z_tt = pd.DataFrame(data_zs5, columns = ['z_supply'])
z_raised = pd.DataFrame(data_zs, columns = ['z_raised'])
zprice_s1 = pd.DataFrame(data_zs6, columns = ['zprice_s1'])
zprice_s2 = pd.DataFrame(data_zs7, columns = ['zprice_s2'])
zprice_s3 = pd.DataFrame(data_zs8, columns = ['zprice_s3'])
zprice_s4 = pd.DataFrame(data_zs9, columns = ['zprice_s4'])
zprice_s5 = pd.DataFrame(data_zs10, columns = ['zprice_s5'])
zprice_s6 = pd.DataFrame(data_zs11, columns = ['zprice_s6'])
zprice_s7 = pd.DataFrame(data_zs12, columns = ['zprice_s7'])
zprice_s8 = pd.DataFrame(data_zs13, columns = ['zprice_s8'])
# zvolume_s1 = pd.DataFrame(volume_zs1, columns = ['zvolume_s1'])
# zvolume_s2 = pd.DataFrame(volume_zs2, columns = ['zvolume_s2'])
# zvolume_s3 = pd.DataFrame(volume_zs3, columns = ['zvolume_s3'])
# zvolume_s4 = pd.DataFrame(volume_zs4, columns = ['zvolume_s4'])
# zvolume_s5 = pd.DataFrame(volume_zs5, columns = ['zvolume_s5'])
# zvolume_s6 = pd.DataFrame(volume_zs6, columns = ['zvolume_s6'])
# zvolume_s7 = pd.DataFrame(volume_zs7, columns = ['zvolume_s7'])
# zvolume_s8 = pd.DataFrame(volume_zs8, columns = ['zvolume_s8'])
print('OK')

OK


##### winsorize

In [8]:
# win_z_season1 = pd.DataFrame(winsorize(z_season1.values, limits = [0.1, 0.1]),columns = ['win_z_season1'])
# win_z_season2 = pd.DataFrame(winsorize(z_season2.values, limits = [0.1, 0.1]),columns = ['win_z_season2'])
# win_z_season3 = pd.DataFrame(winsorize(z_season3.values, limits = [0.1, 0.1]),columns = ['win_z_season3'])
# win_z_season4 = pd.DataFrame(winsorize(z_season4.values, limits = [0.1, 0.1]),columns = ['win_z_season4'])

win_zprice_s1 = pd.DataFrame(winsorize(zprice_s1.values, limits = [0.1, 0.1]),columns = ['win_zprice_s1'])
win_zprice_s2 = pd.DataFrame(winsorize(zprice_s2.values, limits = [0.1, 0.1]),columns = ['win_zprice_s2'])
win_zprice_s3 = pd.DataFrame(winsorize(zprice_s3.values, limits = [0.1, 0.1]),columns = ['win_zprice_s3'])
win_zprice_s4 = pd.DataFrame(winsorize(zprice_s4.values, limits = [0.1, 0.1]),columns = ['win_zprice_s4'])
win_zprice_s5 = pd.DataFrame(winsorize(zprice_s1.values, limits = [0.1, 0.1]),columns = ['win_zprice_s5'])
win_zprice_s6 = pd.DataFrame(winsorize(zprice_s2.values, limits = [0.1, 0.1]),columns = ['win_zprice_s6'])
win_zprice_s7 = pd.DataFrame(winsorize(zprice_s3.values, limits = [0.1, 0.1]),columns = ['win_zprice_s7'])
win_zprice_s8 = pd.DataFrame(winsorize(zprice_s4.values, limits = [0.1, 0.1]),columns = ['win_zprice_s8'])

print('OK')

OK


##### dummy variable

In [30]:
data_work = pd.get_dummies(df2['work'], prefix = 'work')
data_department = pd.get_dummies(df2['department'], prefix = 'department')
data_degree = pd.get_dummies(df2['degree'], prefix = 'degree')
data_kyc = pd.get_dummies(df2['KYC'], prefix = 'KYC')
data_female = pd.get_dummies(df2['Female'], prefix = 'Female')
data_nonwhite = pd.get_dummies(df2['Non_white'], prefix = 'Non_white')
data_raised = pd.get_dummies(df2['Raised'], prefix = 'Raised')
data_google = pd.get_dummies(df2['google_trend2'], prefix = 'google')

In [31]:
df3 = pd.concat([df2, z_tt, z_raised,
                # z_season1, z_season2, z_season3, z_season4, win_z_season1, win_z_season2, win_z_season3, win_z_season4,
                zprice_s1, zprice_s2, zprice_s3, zprice_s4, zprice_s5, zprice_s6, zprice_s7, zprice_s8,
                # zvolume_s1, zvolume_s2, zvolume_s3, zvolume_s4, zvolume_s5, zvolume_s6, zvolume_s7, zvolume_s8,
                # win_zprice_s5, win_zprice_s6, win_zprice_s7, win_zprice_s8,
                data_work, data_department, data_degree, data_kyc, data_female, data_nonwhite, data_raised, data_google], axis = 1)
# df3.columns
print('OK')

OK


##### 類別變數

In [83]:
data_nonwhite = pd.Categorical(values = df2['Non_white'], categories = [0, 1], ordered = False)
data_female = pd.Categorical(values = df2['Female'], categories = [0, 1], ordered = False)
data_kyc = pd.Categorical(values = df2['KYC'], categories = [0, 1],  ordered = False)
data_degree = pd.Categorical(values = df2['degree'], categories = [0, 1],  ordered = False)
data_department = pd.Categorical(values = df2['department'], categories = [0, 1],  ordered = False)
data_work = pd.Categorical(values = df2['work'], categories = [0, 1],  ordered = False)

data_work

[1, 0, 1, 0, 0, ..., 0, 0, 0, 0, 0]
Length: 201
Categories (2, int64): [0, 1]

In [ ]:
data_work = pd.get_dummies(df2['work'], prefix = 'work')
data_work

##### 合併所有column

In [35]:
df3 = pd.concat([df2, z_tt, z_raised,
                # z_season1, z_season2, z_season3, z_season4, win_z_season1, win_z_season2, win_z_season3, win_z_season4,
                zprice_s1, zprice_s2, zprice_s3, zprice_s4, zprice_s5, zprice_s6, zprice_s7, zprice_s8,
                # zvolume_s1, zvolume_s2, zvolume_s3, zvolume_s4, zvolume_s5, zvolume_s6, zvolume_s7, zvolume_s8,
                win_zprice_s5, win_zprice_s6, win_zprice_s7, win_zprice_s8], axis = 1)
# data_work, data_department, data_degree, data_kyc, data_female, data_nonwhite
# df3.columns
print('OK')

OK


##### 用年份及國家當index

In [33]:
df4 = df3 #.set_index(['country', 'Date']).sort_index(level = 'country')
df4.head()

,ICO Name,TICKER,names,Founder,google_trend,google_trend2,degree,degree_2,department,work,...,KYC_0,KYC_1,Female_0,Female_1,Non_white_0,Non_white_1,Raised_0.0,Raised_1.0,google_0.0,google_1.0
0,airbloc,ABL,airbloc,Sungpil Nam,0,0.0,0,1.0,1,1,...,0,1,1,0,0,1,0,1,1,0
1,arbidex,ABX,arbidex,Kamilya Arslanova,1,0.0,0,1.0,0,0,...,1,0,0,1,1,0,1,0,1,0
2,the-abyss,ABYSS,abyss,Konstantin Boyko-Romanovsky,0,0.0,1,2.0,0,1,...,0,1,1,0,1,0,1,0,1,0
3,adbank,ADB,adbank,Jon Gillham,1,1.0,0,1.0,1,1,...,1,0,1,0,1,0,0,1,0,1
4,adshares,ADST,adshares,Jacek Zem?o,0,0.0,0,1.0,1,0,...,1,0,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,welltrado,WTL,welltrado,Tomas Medeckis,0,0.0,1,2.0,0,0,...,0,1,1,0,1,0,0,1,1,0
186,x8-project,X8X,x8x token,Gregor Ko鱉elj,0,0.0,0,1.0,0,0,...,1,0,1,0,1,0,0,1,1,0
187,xriba,XRA,xriba,Gianluca Massini Rosati,1,1.0,0,1.0,0,0,...,0,1,1,0,1,0,0,1,0,1
188,xyo-network,XYO,xyo,Arie Trouw,1,0.0,0,1.0,1,1,...,0,1,1,0,1,0,0,1,1,0


##### 迴歸

In [ ]:
series_dict = {'FWHR':df2['FWHR'], 'Trust':df2['Trust'], 'Dom':df2['Dom'], 'Attr':df2['Attr'] ,'teamsize':df2['Teamsize'],
               'followers':np.log(df2['Followers']), 'non_white':data_nonwhite, 'female':data_female, 'total_supply':np.log(df2['Total supply']),
               'kyc':data_kyc, 'y':np.log(df2['Raised'])}
# series_dict = {'FWHR':df2['FWHR'], 'Trust':df2['Trust'], 'Dom':df2['Dom'], 'Attr':df2['Attr'] ,
#                'non_white':data_nonwhite, 'female':data_female, 'total_supply':tt, 'y':yy}
df3 = pd.DataFrame(series_dict)
X = df3[['FWHR', 'Trust', 'Dom', 'Attr', 'non_white', 'female', 'total_supply', 'followers', 'kyc']]  # 
y = df3[['y']]
regr = linear_model.LinearRegression()
regr.fit(X, y)
df3

In [ ]:
print('R-square :', regr.score(X, y))
print('Coefficient :', regr.coef_)
print('Intercept :', regr.intercept_)
# print(regr.predict)

##### OLS

In [34]:
series_dict = {
        'FWHR':df4['FWHR'], 'Trust':df4['Trust'], 'Dom':df4['Dom'], 'Attr':df4['Attr'] ,'teamsize':df4['Teamsize'],
        'followers':np.log(df4['Followers']),'duration':df4['duration'] / 30, 'kyc_1':df4['KYC_1'],
        'z_supply':df4['z_supply'], 'Raised':np.log1p(df4['Raised']), 'z_raised':df4['z_raised'], 'total_supply':np.log1p(df4['Total supply']),
        'price_season1':df4['price_season1'], 'price_season2':df4['price_season2'], 'price_season3':df4['price_season3'], 'price_season4':df4['price_season4'],
        'price_season5':df4['price_season5'], 'price_season6':df4['price_season6'], 'price_season7':df4['price_season7'], 'price_season8':df4['price_season8'],
        'zprice_s1':df4['zprice_s1'], 'zprice_s2':df4['zprice_s2'], 'zprice_s3':df4['zprice_s3'], 'zprice_s4':df4['zprice_s4'],
        'zprice_s5':df4['zprice_s5'], 'zprice_s6':df4['zprice_s6'], 'zprice_s7':df4['zprice_s7'], 'zprice_s8':df4['zprice_s8'],
        '1d_return':df4['1d_return'], '3m_return':df4['3m_return'], '6m_return':df4['6m_return'],
        '1y_return':df4['1y_return'], '18m_return':df4['18m_return'], '2y_return':df4['2y_return'],
        'd1_excess':df4['1d_excess'], '3m_market':df4['3m_market'], '6m_market':df4['6m_market'], '1y_market':df4['1y_market'],
        '18m_market':df4['18m_market'], '2y_market':df4['2y_market'], 'd1_excess2':df4['1d_excess2'],
        'btc_s1':df4['btc_s1'], 'btc_s2':df4['btc_s2'], 'btc_s3':df4['btc_s3'], 'btc_s4':df4['btc_s4'],
        'btc_s5':df4['btc_s5'], 'btc_s6':df4['btc_s6'], 'btc_s7':df4['btc_s7'], 'btc_s8':df4['btc_s8'],
        'btc_s1_mu':df4['btc_s1_mu'], 'btc_s2_mu':df4['btc_s2_mu'], 'btc_s3_mu':df4['btc_s3_mu'], 'btc_s4_mu':df4['btc_s4_mu'],
        'btc_s5_mu':df4['btc_s5_mu'], 'btc_s6_mu':df4['btc_s6_mu'], 'btc_s7_mu':df4['btc_s7_mu'], 'btc_s8_mu':df4['btc_s8_mu'],
        'non_white_1':df4['Non_white_1'], 'female_1':df4['Female_1'],'degree_1':df4['degree_1'], 'department_1':df4['department_1'],
        'work_1':df4['work_1'], 'Raised_1':df4['Raised_1.0'], 'google_1':df4['google_1.0'],
        'trust*google':df4['Trust']*df4['google_trend2'], 'trust*raised':df4['Trust']*df4['Raised'],
        'attr*google':df4['Attr']*df4['google_trend2'], 'dom*google':df4['Dom']*df4['google_trend2']
        }

In [51]:
df5 = pd.DataFrame(series_dict)
X = df5[['Attr', 'Trust', 'Dom', 'non_white_1', 'female_1', 'total_supply', 'teamsize', 'followers', 'kyc_1', 'duration', 'd1_excess',
        'degree_1', 'work_1', 'department_1', '1y_market']]
XX = df5[['Attr', 'google_1', 'trust*google', 'btc_s5_mu', 'Trust', 'Dom', 'non_white_1', 'female_1', 'total_supply', 'teamsize', 'followers', 'kyc_1', 'duration', 'd1_excess',
        'degree_1', 'work_1', 'department_1']]  # , '1y_market'
y = df5['price_season5']
 
# 建立迴歸模型
# X1 = sm.add_constant(X)
X2 = sm.add_constant(XX)
# linear_model2 = sm.OLS(endog = y, exog = X1, missing = 'drop').fit()
linear_model3 = sm.OLS(endog = y, exog = X2, missing = 'drop').fit()

# print(linear_model2.summary())

# print('*' * 100)
# print('*' * 100)
# print('*' * 100)

print(linear_model3.summary())

# 加入固定效應控制貨幣上市年

# exog = sm.add_constant(X)
# exog2 = sm.add_constant(XX)
# ico = PanelOLS(y, exog, entity_effects = False, time_effects = True)
# ico2 = PanelOLS(y, exog2, entity_effects = False, time_effects = True)
# res = ico.fit()
# res2 = ico2.fit()
# print(res)
# print('*' * 100)
# print('*' * 100)
# print('*' * 100)
# print(res2)

                            OLS Regression Results                            
Dep. Variable:          price_season5   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1.564
Date:                Wed, 22 Mar 2023   Prob (F-statistic):             0.0787
Time:                        17:56:27   Log-Likelihood:                -619.90
No. Observations:                 190   AIC:                             1276.
Df Residuals:                     172   BIC:                             1334.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           12.1373      5.872      2.067   

##### 固定效應範例

In [194]:
from statsmodels.datasets import grunfeld
data = grunfeld.load_pandas().data
data = data.set_index(["firm","year"])
print(data.head())

                       invest   value  capital
firm           year                           
General Motors 1935.0   317.6  3078.5      2.8
               1936.0   391.8  4661.7     52.6
               1937.0   410.6  5387.1    156.9
               1938.0   257.7  2792.2    209.2
               1939.0   330.8  4313.2    203.4


In [ ]:
# Coding method 1
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
exog = sm.add_constant(data[['value','capital']])
grunfeld_fe = PanelOLS(data['invest'], exog, entity_effects=True, time_effects=False)
grunfeld_fe = grunfeld_fe.fit()
print(grunfeld_fe)

# Coding method 2
# grunfeld_fe = PanelOLS.from_formula("invest ~ value + capital + EntityEffects", data=data)
# print(grunfeld_fe.fit())

In [ ]:
from linearmodels.panel import PanelOLS
import pandas as pd
from linearmodels.datasets import wage_panel
import statsmodels.api as sm

data = wage_panel.load()
year = pd.Categorical(data.year)
data = data.set_index(["nr", "year"])
# data["year"] = year

exog_vars = ["expersq", "union", "married"]#, "year"
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, time_effects = True)
fe_res = mod.fit()
print(fe_res)

In [157]:
# df4['Date'] = pd.Categorical(df4.Date)
FE_ols = smf.ols(formula = 'lwage ~ expersq + union + married', data = data).fit()#  + year
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     106.4
Date:                Fri, 10 Mar 2023   Prob (F-statistic):           1.70e-66
Time:                        15:40:08   Log-Likelihood:                -3285.2
No. Observations:                4360   AIC:                             6578.
Df Residuals:                    4356   BIC:                             6604.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4648      0.014    105.436      0.0

##### 固定效應2

In [92]:
from fixedeffect.iv import iv2sls, ivgmm, ivtest
from fixedeffect.fe import fixedeffect, did, getfe
from fixedeffect.utils.panel_dgp import gen_data

In [ ]:
exog_x = X
y = df5['zprice_s1']
category = df5['Date']
cluster = df5['country']

model_fe = fixedeffect(data_df = X, dependent = y, exog_x = exog_x, category = category, cluster = cluster)
result = model_fe.fit()
result.summary()

##### Logistic Model

In [ ]:
series_dict = {
        'FWHR':df4['FWHR'], 'Trust':df4['Trust'], 'Dom':df4['Dom'], 'Attr':df4['Attr'] ,'teamsize':np.log(df4['Teamsize']), 'female_1':df4['Female_1'],
        'followers':np.log(df4['Followers']), 'non_white_1':df4['Non_white_1'],'duration':df4['duration'] / 30, 'trading':df4['trading'], 
        'z_supply':df4['z_supply'], 'Raised':df4['Raised'], 'z_raised':df4['z_raised'], 'total_supply':np.log1p(df4['Total supply']),
        'price_season1':df4['price_season1'], 'price_season2':df4['price_season2'], 'price_season3':df4['price_season3'], 'price_season4':df4['price_season4'],
        'zprice_s1':df4['zprice_s1'], 'zprice_s2':df4['zprice_s2'], 'zprice_s3':df4['zprice_s3'], 'zprice_s4':df4['zprice_s4'],
        'zprice_s5':df4['zprice_s5'], 'zprice_s6':df4['zprice_s6'], 'zprice_s7':df4['zprice_s7'], 'zprice_s8':df4['zprice_s8'],
        'degree_1':df4['degree_1'], 'department_1':df4['department_1'], 'work_1':df4['work_1'], 'kyc_1':df4['KYC_1'],
        'trust*google':df4['Trust']*df4['google_trend'],'trust*raised':df4['Trust']*df4['Raised']
        }

df5 = pd.DataFrame(series_dict)
X = df5[['Attr', 'Trust', 'Dom', 'non_white_1', 'female_1', 'total_supply', 'teamsize', 'followers', 'kyc_1', 'duration', 'degree_1', 'work_1', 'department_1']]
y = df5['trading']
X1 = sm.add_constant(X)

logiti_reg = linear_model.LogisticRegression()
logiti_reg.fit(X1, y)

In [78]:
series_dict = {
        'FWHR':df4['FWHR'], 'Trust':df4['Trust'], 'Dom':df4['Dom'], 'Attr':df4['Attr'] ,'teamsize':np.log(df4['Teamsize']), 'female_1':df4['Female_1'],
        'followers':np.log(df4['Followers']), 'non_white_1':df4['Non_white_1'],'duration':df4['duration'] / 30, 'trading':df4['trading'], 
        'z_supply':df4['z_supply'], 'Raised':df4['Raised'], 'z_raised':df4['z_raised'], 'total_supply':np.log1p(df4['Total supply']),
        'price_season1':df4['price_season1'], 'price_season2':df4['price_season2'], 'price_season3':df4['price_season3'], 'price_season4':df4['price_season4'],
        'zprice_s1':df4['zprice_s1'], 'zprice_s2':df4['zprice_s2'], 'zprice_s3':df4['zprice_s3'], 'zprice_s4':df4['zprice_s4'],
        'zprice_s5':df4['zprice_s5'], 'zprice_s6':df4['zprice_s6'], 'zprice_s7':df4['zprice_s7'], 'zprice_s8':df4['zprice_s8'],
        'degree_1':df4['degree_1'], 'department_1':df4['department_1'], 'work_1':df4['work_1'], 'kyc_1':df4['KYC_1'],
        'trust*google':df4['Trust']*df4['google_trend'],'trust*raised':df4['Trust']*df4['Raised']
        }

df5 = pd.DataFrame(series_dict)
X = df5[['Attr', 'Trust', 'Dom', 'non_white_1', 'female_1', 'total_supply', 'teamsize', 'followers', 'kyc_1', 'duration', 'degree_1', 'work_1', 'department_1']]
y = df5['trading']
X1 = sm.add_constant(X)

logiti_reg2 = sm.Logit(endog = y, exog = X1).fit()
# logiti_reg.fit(X1, y)

Optimization terminated successfully.
         Current function value: 0.587715
         Iterations 6


In [ ]:
print(logiti_reg2.summary())

In [76]:
logiti_reg.coef_

array([[-0.31771797,  0.37942227, -0.06400823, -0.92174912,  0.08694817,
         0.70254466,  0.02998852,  0.40792196,  0.07509724, -0.01567601,
        -0.34417841,  0.05910139,  0.64918388,  0.0893804 ]])

In [ ]:
# data aaa;
# set aa;
# dd = duration/30;
# follow = Log(followers);
# team = Log(teamsize);
# supply = Log(total_supply+1);
# run;
# proc logistic data = aaa;
# class google_trend degree department work non_white female kyc/;
# model trading = trust dom attr degree department work non_white female kyc dd follow team supply;
# run;

##### 處理市值檔案

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
# workbook.save('C:/Users/user/Desktop/month_return.xlsx')

wb = openpyxl.load_workbook('C:/Users/user/Desktop/month_return.xlsx', data_only = True)
s1 = wb['Sheet']
s1.cell(1, 1).value = 'Name'
s1.cell(2, 1).value = 'Drep'

path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Drep'
fnames = glob.glob(path_ + r'\*.csv', recursive = False)
df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
# 去除市值為0的日子
df = df[df['marketCap'] != 0]
# 重設index
df.reset_index(drop = True, inplace = True)
# 算市值成長率
month_data = []
month_mean = []
for j in range(0, len(df['marketCap']), 31):
    month_data.append(df['marketCap'][j:j+30])
for k in range(0, len(month_data)):
    if len(month_data[k]) == 30:
        month_mean.append(month_data[k].sum()/30)
    else:
        month_mean.append(month_data[k].sum() / len(month_data[len(month_data)-1]))
    s1.cell(2, k + 2).value = month_mean[k]
wb.save('C:/Users/user/Desktop/month_return.xlsx')

In [ ]:
# workbook = openpyxl.Workbook()
# sheet = workbook.worksheets[0]
# workbook.save('C:/Users/user/Desktop/return_rate.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

# path1 = 'C:/Users/user/Desktop/return_rate.xlsx'
# df_none = pd.DataFrame()
# if not os.path.isfile(path1):
#     df_none.to_excel(path1)

# 讀入CMC抓取的市值並合併
wb = openpyxl.load_workbook('C:/Users/user/Desktop/return_rate.xlsx', data_only = True)
s1 = wb['Sheet1']
s1['A1'].value = 'name'
s1['B1'].value = 'return180'
s1['C1'].value = 'return365'
s1['D1'].value = 'return730'

for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['marketCap'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    # 算市值成長率
    if len(df) < 180:
        s1.cell(i + 2, 1).value = dirname[i]
        wb.save('C:/Users/user/Desktop/return_rate.xlsx')
        continue
    elif 180 <= len(df) < 365:
        trading_0 = df.iloc[0, :]['marketCap']
        trading_180 = df.iloc[180, :]['marketCap']
        return_rate180 = (trading_180 - trading_0)/ trading_180
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = return_rate180
        wb.save('C:/Users/user/Desktop/return_rate.xlsx')
    elif 365 <= len(df) < 730:
        trading_0 = df.iloc[0, :]['marketCap']
        trading_180 = df.iloc[180, :]['marketCap']
        trading_365 = df.iloc[365, :]['marketCap']
        return_rate180 = (trading_180 - trading_0)/ trading_180
        return_rate365 = (trading_365 - trading_0)/ trading_365
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = return_rate180
        s1.cell(i + 2, 3).value = return_rate365
        wb.save('C:/Users/user/Desktop/return_rate.xlsx')
    else:
        trading_0 = df.iloc[0, :]['marketCap']
        trading_180 = df.iloc[180, :]['marketCap']
        trading_365 = df.iloc[365, :]['marketCap']
        trading_730 = df.iloc[730, :]['marketCap']
        return_rate180 = (trading_180 - trading_0)/ trading_180
        return_rate365 = (trading_365 - trading_0)/ trading_365
        return_rate730 = (trading_730 - trading_0)/ trading_730
        # data = [dirname[i], return_rate180, return_rate365, return_rate730]
        # s1.append(data)
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = return_rate180
        s1.cell(i + 2, 3).value = return_rate365
        s1.cell(i + 2, 4).value = return_rate730
        wb.save('C:/Users/user/Desktop/return_rate.xlsx')
    # print(i)

In [ ]:
# workbook = openpyxl.Workbook()
# sheet = workbook.worksheets[0]
# workbook.save(r'C:\Users\user\Desktop\coingecko_return.xlsx')

cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook(r'C:\Users\user\Desktop\coingecko_return.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'name'
s1['B1'].value = 'return180'
s1['C1'].value = 'return365'
s1['D1'].value = 'return730'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    df3 = df3[df3['marketcap'] != 0]
    df3.reset_index(drop = True, inplace = True)
    # 算市值成長率
    if len(df3) < 180:
        s1.cell(i + 2, 1).value = name
        wb.save('C:/Users/user/Desktop/coingecko_return.xlsx')
        continue
    elif 180 <= len(df3) < 365:
        trading_0 = df3.iloc[0, :]['marketcap']
        trading_180 = df3.iloc[180, :]['marketcap']
        return_rate180 = (trading_180 - trading_0)/ trading_180
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = return_rate180
        wb.save('C:/Users/user/Desktop/coingecko_return.xlsx')
    elif 365 <= len(df3) < 730:
        trading_0 = df3.iloc[0, :]['marketcap']
        trading_180 = df3.iloc[180, :]['marketcap']
        trading_365 = df3.iloc[365, :]['marketcap']
        return_rate180 = (trading_180 - trading_0)/ trading_180
        return_rate365 = (trading_365 - trading_0)/ trading_365
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = return_rate180
        s1.cell(i + 2, 3).value = return_rate365
        wb.save('C:/Users/user/Desktop/coingecko_return.xlsx')
    else:
        trading_0 = df3.iloc[0, :]['marketcap']
        trading_180 = df3.iloc[180, :]['marketcap']
        trading_365 = df3.iloc[365, :]['marketcap']
        trading_730 = df3.iloc[730, :]['marketcap']
        return_rate180 = (trading_180 - trading_0)/ trading_180
        return_rate365 = (trading_365 - trading_0)/ trading_365
        return_rate730 = (trading_730 - trading_0)/ trading_730
        # data = [dirname[i], return_rate180, return_rate365, return_rate730]
        # s1.append(data)
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = return_rate180
        s1.cell(i + 2, 3).value = return_rate365
        s1.cell(i + 2, 4).value = return_rate730
        wb.save(r'C:\Users\user\Desktop\coingecko_return.xlsx')

##### 市值月資料

In [ ]:
# workbook = openpyxl.Workbook()
# sheet = workbook.worksheets[0]
# workbook.save('C:/Users/user/Desktop/ICOdata/month_cap.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

# path1 = 'C:/Users/user/Desktop/return_rate.xlsx'
# df_none = pd.DataFrame()
# if not os.path.isfile(path1):
#     df_none.to_excel(path1)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/ICOdata/month_cap.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Month_{n}'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i]
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['marketCap'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    month_data = []
    month_mean = []
    month_rate = []
    # 將資料每個月分成一組
    for j in range(0, len(df['marketCap']), 31):
        month_data.append(df['marketCap'][j:j+30])
    # 每組(每個月)市值加總算平均
    for k in range(0, len(month_data)):
        if len(month_data[k]) == 30:
            month_mean.append(month_data[k].sum()/30)
        else:
            month_mean.append(month_data[k].sum() / len(month_data[len(month_data)-1]))
    # # 計算市值月成長率
    # for m in range(0, len(month_mean)-1):
    #     month_rate.append((month_mean[m+1] - month_mean[m]) / month_mean[m])
        s1.cell(i + 2, k + 2).value = month_mean[k]

    wb.save('C:/Users/user/Desktop/ICOdata/month_cap.xlsx')
    # print(i)

In [ ]:
# workbook = openpyxl.Workbook()
# sheet = workbook.worksheets[0]
# workbook.save('C:/Users/user/Desktop/ICOdata/coingecko_month_cap.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/ICOdata/coingecko_month_cap.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Month_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name
    df3 = df3[df3['marketcap'] != 0]
    df3.reset_index(drop = True, inplace = True)
    month_data = []
    month_mean = []
    # 將資料每個月分成一組
    for j in range(0, len(df3['marketcap']), 31):
        month_data.append(df3['marketcap'][j:j+30])
    # 每組(每個月)市值加總算平均
    for k in range(0, len(month_data)):
        if len(month_data[k]) == 30:
            month_mean.append(month_data[k].sum()/30)
        else:
            month_mean.append(month_data[k].sum() / len(month_data[len(month_data)-1]))
        s1.cell(i + 2, k + 2).value = month_mean[k]

    wb.save('C:/Users/user/Desktop/ICOdata/coingecko_month_cap.xlsx')

##### 市值季資料

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/CMC市值季資料2.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

# path1 = 'C:/Users/user/Desktop/return_rate.xlsx'
# df_none = pd.DataFrame()
# if not os.path.isfile(path1):
#     df_none.to_excel(path1)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/CMC市值季資料2.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i]
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['marketCap'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    season_data = []
    season_mean = []
    season_rate = []
    # 將資料每季分成一組
    for j in range(0, len(df['marketCap']), 91):
        season_data.append(df['marketCap'][j:j+90])
    # 每組(每季)市值加總算平均
    for k in range(0, len(season_data)):
        if len(season_data[k]) == 90:
            season_mean.append(season_data[k].sum()/90)
        else:
            season_mean.append(season_data[k].sum() / len(season_data[len(season_data)-1]))
    # # 計算市值月成長率
    # for m in range(0, len(month_mean)-1):
    #     month_rate.append((month_mean[m+1] - month_mean[m]) / month_mean[m])
        s1.cell(i + 2, k + 2).value = season_mean[k]

    wb.save('C:/Users/user/Desktop/CMC市值季資料2.xlsx')
    # print(i)

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/gecko市值季資料.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/gecko市值季資料.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name
    df3 = df3[df3['marketcap'] != 0]
    df3.reset_index(drop = True, inplace = True)
    season_data = []
    season_mean = []
    # 將資料每季分成一組
    for j in range(0, len(df3['marketcap']), 91):
        season_data.append(df3['marketcap'][j:j+90])
    # 每組(每季)市值加總算平均
    for k in range(0, len(season_data)):
        if len(season_data[k]) == 90:
            season_mean.append(season_data[k].sum()/90)
        else:
            season_mean.append(season_data[k].sum() / len(season_data[len(season_data)-1]))
        s1.cell(i + 2, k + 2).value = season_mean[k]

    wb.save('C:/Users/user/Desktop/gecko市值季資料.xlsx')

##### 貨幣上市年

In [158]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/CMC貨幣上市年.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/CMC貨幣上市年.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
s1.cell(1, 2).value = 'Date'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i]
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['high'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    s1.cell(i + 2, 2).value = df['Date'][0][0:4]
    wb.save('C:/Users/user/Desktop/CMC貨幣上市年.xlsx')
 # print(i)

In [159]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/gecko貨幣上市年.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/gecko貨幣上市年.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
s1.cell(1, 2).value = 'Date'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)
    
    s1.cell(i + 2, 2).value = df3['date'][0][0:4]

    wb.save('C:/Users/user/Desktop/gecko貨幣上市年.xlsx')

##### 價格季資料

In [19]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/1d_price.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)


# path1 = 'C:/Users/user/Desktop/return_rate.xlsx'
# df_none = pd.DataFrame()
# if not os.path.isfile(path1):
#     df_none.to_excel(path1)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/1d_price.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i].lower()
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['close'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    s1.cell(i + 2, 2).value = df['close'][0]
    # season_data = []
    # season_mean = []
    # season_rate = []
    # 將資料每季分成一組
    # for j in range(0, len(df['close']), 90):
    #     season_data.append(df['close'][j:j+90])

    # (季平均-交易第一天)/交易第一天
    # for k in range(0, len(season_data)-1):
    #     if len(season_data[k]) == 90:
    #         season_mean.append((season_data[k].sum()/90 - df['close'][0])/df['close'][0])
    #     else:
    #         season_mean.append(((season_data[k].sum() / len(season_data[len(season_data)-1])) - df['high'][0])/df['high'][0])
    
    # (後一季-前一季)/前一季
    # for k in range(0, len(season_data)-1):
    #     if len(season_data[k]) == 90:
    #         season_mean.append((season_data[k+1].sum()/90 - season_data[k][90*k])/(season_data[k][90*k]))
    #     else:
    #         season_mean.append(((season_data[k+1].sum() / len(season_data[len(season_data)-1])) - season_data[k][(len(season_data)-1)*90])/(season_data[k][(len(season_data)-1)*90]))
    
    # 每組(每季)價格加總算平均
    # for k in range(0, len(season_data)):
    #     if len(season_data[k]) == 90:
    #         season_mean.append((season_data[k][90*(k+1)-1] - season_data[k][90*k])/(season_data[k][90*k]))
    #     else:
    #         season_mean.append((season_data[k][(len(season_data)-1)*90 + len(season_data[k]) - 1] - season_data[k][(len(season_data)-1)*90])/(season_data[k][(len(season_data)-1)*90]))

    # 計算價格成長率
    # for m in range(0, len(month_mean)-1):
    #     month_rate.append((month_mean[m+1] - month_mean[m]) / month_mean[m])
        # s1.cell(i + 2, k + 2).value = season_mean[k]

    wb.save('C:/Users/user/Desktop/1d_price.xlsx')
    # print(i)

In [20]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/gecko_d1_price.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/gecko_d1_price.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
s1['B1'].value = 'd1_price'
# for n in range(1, 65):
#     s1.cell(1, n + 1).value = f'Season_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name.lower()
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)
    s1.cell(i + 2, 2).value = df3['prices'][0]
    # season_data = []
    # season_mean = []
    # 將資料每季分成一組
    # for j in range(0, len(df3['prices']), 90):
    #     season_data.append(df3['prices'][j:j+90])

    # (季平均-交易第一天)/交易第一天
    # for k in range(0, len(season_data)-1):
    #     if len(season_data[k]) == 90:
    #         season_mean.append((season_data[k].sum()/90 - df3['prices'][0])/df3['prices'][0])
    #     else:
    #         season_mean.append(((season_data[k].sum() / len(season_data[len(season_data)-1])) - df3['prices'][0])/df3['prices'][0])

    # 每組(每季)市值加總算平均
    # for k in range(0, len(season_data)-1):
    #     if len(season_data[k]) == 90:
    #         season_mean.append((season_data[k+1].sum()/90 - season_data[k][90*k])/(season_data[k][90*k]))
    #     else:
    #         season_mean.append(((season_data[k+1].sum() / len(season_data[len(season_data)-1])) - season_data[k][(len(season_data)-1)*90])/(season_data[k][(len(season_data)-1)*90]))
   
    # for k in range(0, len(season_data)):
    #     if len(season_data[k]) == 90:
    #         season_mean.append((season_data[k][90*(k+1)-1] - season_data[k][90*k])/(season_data[k][90*k]))
    #     else:
    #         season_mean.append((season_data[k][(len(season_data)-1)*90 + len(season_data[k]) - 1] - season_data[k][(len(season_data)-1)*90])/(season_data[k][(len(season_data)-1)*90]))
    # for k in range(0, len(season_data)):
    #     if len(season_data[k]) == 90:
    #         season_mean.append(season_data[k].sum()/90)
    #     else:
    #         season_mean.append(season_data[k].sum() / len(season_data[len(season_data)-1]))
        # s1.cell(i + 2, k + 2).value = season_mean[k]

    wb.save('C:/Users/user/Desktop/gecko_d1_price.xlsx')

##### 3、6、12 month return

In [ ]:
path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames = glob.glob(path_ + r'\*.csv', recursive = False)
df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
# 去除市值為0的日子
df = df[df['close'] != 0]
# 重設index
df.reset_index(drop = True, inplace = True)
df

In [57]:
aa = df[df['Date'] == '2022-09-25'].index[0]
df['close'][aa]

18802.0979756765

In [40]:
path2_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Ethereum'
fnames2 = glob.glob(path2_ + r'\*.csv', recursive = False)
df_list2 = [pd.read_csv(fname2, sep = ',', encoding = 'utf-8', engine = 'python') for fname2 in fnames2]
df2 = pd.concat(df_list2, axis = 0, verify_integrity = True, ignore_index = True)
# 去除市值為0的日子
df2 = df2[df2['close'] != 0]
# 重設index
df2.reset_index(drop = True, inplace = True)

In [50]:
print(df['marketCap'][0] / df2['marketCap'][0])
print(df['marketCap'][90] / df2['marketCap'][90])
print(df['marketCap'][181] / df2['marketCap'][181])
print(df['marketCap'][364] / df2['marketCap'][364])

35.25782296786226
16.587342416279576
10.665641097891937
8.748868383066354


In [297]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/month_return2.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/month_return2.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
s1['B1'].value = '18m_return'
s1['C1'].value = '2y_return'
# s1['D1'].value = '1y_return'
# s1['E1'].value = '1d_return'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i].lower()
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['close'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)

    if len(df['close']) < 730:
        continue
    else:
        eighteen_month = (df['close'][545] - df['close'][1])/df['close'][1]
        two_year = (df['close'][730] - df['close'][1])/df['close'][1]
        # one_year = (df['close'][364] - df['close'][1])/df['close'][1]
        # one_day = (df['close'][1] - df['close'][0])/df['close'][0]
        s1.cell(i + 2, 2).value = eighteen_month
        s1.cell(i + 2, 3).value = two_year
        # s1.cell(i + 2, 4).value = one_year
        # s1.cell(i + 2, 5).value = one_day

    wb.save('C:/Users/user/Desktop/month_return2.xlsx')

In [298]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/month_return3.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/month_return3.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
s1['B1'].value = '18m_return'
s1['C1'].value = '2y_return'
# s1['D1'].value = '1y_return'
# s1['E1'].value = '1d_return'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name.lower()
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)

    if len(df3['prices']) < 364:
        continue
    else:
        eighteen_month = (df3['prices'][90] - df3['prices'][1])/df3['prices'][1]
        two_year = (df3['prices'][181] - df3['prices'][1])/df3['prices'][1]
        # one_year = (df3['prices'][364] - df3['prices'][1])/df3['prices'][1]
        # one_day = (df3['prices'][1] - df3['prices'][0])/df3['prices'][0]
        s1.cell(i + 2, 2).value = eighteen_month
        s1.cell(i + 2, 3).value = two_year
        # s1.cell(i + 2, 4).value = one_year
        # s1.cell(i + 2, 5).value = one_day

    wb.save('C:/Users/user/Desktop/month_return3.xlsx')

##### 比特幣季回報

In [69]:
path1_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames1 = glob.glob(path1_ + r'\*.csv', recursive = False)
df_list1 = [pd.read_csv(fname1, sep = ',', encoding = 'utf-8', engine = 'python') for fname1 in fnames1]
ddf = pd.concat(df_list1, axis = 0, verify_integrity = True, ignore_index = True)
ddf = ddf[ddf['close'] != 0]
ddf.reset_index(drop = True, inplace = True)

workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/bitcoin_season.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/bitcoin_season.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i].lower()
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['close'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    season_data = []
    bitcoin_rate = []
    # 將資料每季分成一組
    for j in range(0, len(df), 90):
        season_data.append(df[j:j+90].reset_index(drop = True, inplace = False))

    for k in range(0, len(season_data)-1):
        if len(season_data[k]) == 90:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][0])].index[0]
            ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][89])].index[0]
            bitcoin_rate.append((ddf['close'][ss2]-ddf['close'][ss1])/ddf['close'][ss1])
        else:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][0])].index[0]
            ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][len(season_data)[k]])].index[0]
            bitcoin_rate.append((ddf['close'][ss2]-ddf['close'][ss1])/ddf['close'][ss1])
    
        s1.cell(i + 2, k + 2).value = bitcoin_rate[k]

    wb.save('C:/Users/user/Desktop/bitcoin_season.xlsx')

In [73]:
path1_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames1 = glob.glob(path1_ + r'\*.csv', recursive = False)
df_list1 = [pd.read_csv(fname1, sep = ',', encoding = 'utf-8', engine = 'python') for fname1 in fnames1]
ddf = pd.concat(df_list1, axis = 0, verify_integrity = True, ignore_index = True)
ddf = ddf[ddf['close'] != 0]
ddf.reset_index(drop = True, inplace = True)

workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/bitcoin_season2.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/bitcoin_season2.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name.lower()
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)
    season_data = []
    bitcoin_rate = []
    # 將資料每季分成一組
    for j in range(0, len(df3), 90):
        season_data.append(df3[j:j+90].reset_index(drop = True, inplace = False))
    
    for k in range(0, len(season_data)-1):
        if len(season_data[k]) == 90:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][0])].index[0]
            ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][89])].index[0]
            bitcoin_rate.append((ddf['close'][ss2]-ddf['close'][ss1])/ddf['close'][ss1])
        else:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][0])].index[0]
            ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][len(season_data)[k]])].index[0]
            bitcoin_rate.append((ddf['close'][ss2]-ddf['close'][ss1])/ddf['close'][ss1])
    
        s1.cell(i + 2, k + 2).value = bitcoin_rate[k]

    wb.save('C:/Users/user/Desktop/bitcoin_season2.xlsx')

##### 比特幣季平均

In [56]:
path1_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames1 = glob.glob(path1_ + r'\*.csv', recursive = False)
df_list1 = [pd.read_csv(fname1, sep = ',', encoding = 'utf-8', engine = 'python') for fname1 in fnames1]
ddf = pd.concat(df_list1, axis = 0, verify_integrity = True, ignore_index = True)
ddf = ddf[ddf['close'] != 0]
ddf.reset_index(drop = True, inplace = True)

workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/bitcoin_season_mean.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/bitcoin_season_mean.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):  # len(dirname)
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i].lower()
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['close'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    season_data = []
    bitcoin_rate = []
    # 將資料每季分成一組
    for j in range(0, len(df), 90):
        season_data.append(df[j:j+90].reset_index(drop = True, inplace = False))

    for k in range(0, len(season_data)):
        if len(season_data[k]) == 90:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][0])].index[0]
            # ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][89])].index[0]
            bitcoin_rate.append((ddf['close'][ss1:ss1+90].sum())/90)
        else:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][0])].index[0]
            # ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['Date'][len(season_data)[k]])].index[0]
            bitcoin_rate.append((ddf['close'][ss1:ss1+len(season_data[k])].sum())/len(season_data[k]))
    
        s1.cell(i + 2, k + 2).value = bitcoin_rate[k]

    wb.save('C:/Users/user/Desktop/bitcoin_season_mean.xlsx')

In [57]:
path1_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames1 = glob.glob(path1_ + r'\*.csv', recursive = False)
df_list1 = [pd.read_csv(fname1, sep = ',', encoding = 'utf-8', engine = 'python') for fname1 in fnames1]
ddf = pd.concat(df_list1, axis = 0, verify_integrity = True, ignore_index = True)
ddf = ddf[ddf['close'] != 0]
ddf.reset_index(drop = True, inplace = True)

workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/bitcoin_season2_mean.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/bitcoin_season2_mean.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name.lower()
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)
    season_data = []
    bitcoin_rate = []
    # 將資料每季分成一組
    for j in range(0, len(df3), 90):
        season_data.append(df3[j:j+90].reset_index(drop = True, inplace = False))
    
    for k in range(0, len(season_data)):
        if len(season_data[k]) == 90:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][0])].index[0]
            # ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][89])].index[0]
            bitcoin_rate.append((ddf['close'][ss1:ss1+90].sum())/90)
        else:
            ss1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][0])].index[0]
            # ss2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(season_data[k]['date'][len(season_data)[k]])].index[0]
            bitcoin_rate.append((ddf['close'][ss1:ss1+len(season_data)].sum())/len(season_data))
    
        s1.cell(i + 2, k + 2).value = bitcoin_rate[k]

    wb.save('C:/Users/user/Desktop/bitcoin_season2_mean.xlsx')

##### market return

In [322]:
path1_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames1 = glob.glob(path1_ + r'\*.csv', recursive = False)
df_list1 = [pd.read_csv(fname1, sep = ',', encoding = 'utf-8', engine = 'python') for fname1 in fnames1]
ddf = pd.concat(df_list1, axis = 0, verify_integrity = True, ignore_index = True)
# 去除市值為0的日子
ddf = ddf[ddf['close'] != 0]
# 重設index
ddf.reset_index(drop = True, inplace = True)

workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/market_return.xlsx')

# 讀取有抓到市值的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/market_return.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
s1['B1'].value = '18m_return'
s1['C1'].value = '2y_return'
# s1['D1'].value = '1y_return'
# s1['E1'].value = '1d_return'

# 讀入CMC抓取的市值並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i].lower()
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['close'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)

    if len(df['close']) < 730:    # 364
        continue
    else:
        d1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df['Date'][1])].index[0]
        # m3 = ddf[ddf['Date'] == df['Date'][90]].index[0]
        # m6 = ddf[ddf['Date'] == df['Date'][181]].index[0]
        # y1 = ddf[ddf['Date'] == df['Date'][364]].index[0]
        m18 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df['Date'][545])].index[0]
        y2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df['Date'][730])].index[0]
        # three_market = (ddf['close'][m3] - ddf['close'][d1])/ddf['close'][d1]
        # six_market = (ddf['close'][m6] - ddf['close'][d1])/ddf['close'][d1]
        # year_market = (ddf['close'][y1] - ddf['close'][d1])/ddf['close'][d1]
        eighteen_market = (ddf['close'][m18] - ddf['close'][d1])/ddf['close'][d1]
        two_year_market = (ddf['close'][y2] - ddf['close'][d1])/ddf['close'][d1]
        s1.cell(i + 2, 2).value = eighteen_market
        s1.cell(i + 2, 3).value = two_year_market
        # s1.cell(i + 2, 4).value = year_market

    wb.save('C:/Users/user/Desktop/market_return.xlsx')

In [325]:
path1_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/Bitcoin'
fnames1 = glob.glob(path1_ + r'\*.csv', recursive = False)
df_list1 = [pd.read_csv(fname1, sep = ',', encoding = 'utf-8', engine = 'python') for fname1 in fnames1]
ddf = pd.concat(df_list1, axis = 0, verify_integrity = True, ignore_index = True)
# 去除市值為0的日子
ddf = ddf[ddf['close'] != 0]
# 重設index
ddf.reset_index(drop = True, inplace = True)

workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/market_return2.xlsx')

# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/market_return2.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
s1['B1'].value = '18m_return'
s1['C1'].value = '2y_return'
# s1['D1'].value = '1y_return'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name.lower()
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)

    if len(df3['prices']) < 730:    # 364
        continue
    else:
        # d1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df3['date'][1])].index[0]
        # m3 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df3['date'][90])].index[0]
        # m6 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df3['date'][181])].index[0]
        # y1 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df3['date'][364])].index[0]
        m18 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df3['date'][545])].index[0]
        y2 = ddf[pd.to_datetime(ddf['Date']) == pd.to_datetime(df3['date'][730])].index[0]
        # three_market = (ddf['close'][m3] - ddf['close'][d1])/ddf['close'][d1]
        # six_market = (ddf['close'][m6] - ddf['close'][d1])/ddf['close'][d1]
        # year_market = (ddf['close'][y1] - ddf['close'][d1])/ddf['close'][d1]
        eighteen_market = (ddf['close'][m18] - ddf['close'][d1])/ddf['close'][d1]
        two_year_market = (ddf['close'][y2] - ddf['close'][d1])/ddf['close'][d1]
        s1.cell(i + 2, 2).value = eighteen_market
        s1.cell(i + 2, 3).value = two_year_market
        # s1.cell(i + 2, 4).value = year_market

    wb.save('C:/Users/user/Desktop/market_return2.xlsx')

##### coingecko成長率

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/ICOdata/coingecko_return_month.xlsx')

In [ ]:
# 讀取有抓到市值的coin
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook('C:/Users/user/Desktop/ICOdata/coingecko_return_month.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Month_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    s1.cell(i + 2, 1).value = name
    df3 = df3[df3['marketcap'] != 0]
    df3.reset_index(drop = True, inplace = True)
    month_data = []
    month_mean = []
    month_rate = []
    # 將資料每個月分成一組
    for j in range(0, len(df3['marketcap']), 31):
        month_data.append(df3['marketcap'][j:j+30])
    # 每組(每個月)市值加總算平均
    for k in range(0, len(month_data)):
        if len(month_data[k]) == 30:
            month_mean.append(month_data[k].sum()/30)
        else:
            month_mean.append(month_data[k].sum() / len(month_data[len(month_data)-1]))
    # 計算市值月成長率
    for m in range(0, len(month_mean)-1):
        month_rate.append((month_mean[m+1] - month_mean[m]) / month_mean[m])
        s1.cell(i + 2, m + 2).value = month_rate[m]

    wb.save('C:/Users/user/Desktop/ICOdata/coingecko_return_month.xlsx')

##### 市值幾何平均

In [ ]:
df = pd.read_excel('C:/Users/user/Desktop/ICOdata/month_return.xlsx', sheet_name = 'Sheet')
df = df.dropna(axis = 0, subset = ['Month_1', 'Month_12'], how = 'any')
df.reset_index(drop=True, inplace=True)
aa = []
names = []

for i in range(0, len(df)):
    name = df['Name2'][i]
    cagr = 1
    for j in range(1, 13):  
        if j <= 12: 
            cagr = (1 + df[f'Month_{j}'][i]) * cagr
    a = cagr**(1/12) - 1
    aa.append(a)
    names.append(name)
# df['Name2'][10]

In [ ]:
df1 = pd.DataFrame(aa, columns = ['cagr'])
df2 = pd.DataFrame(names, columns = ['Name'])
pd.concat([df2, df1], axis = 1).to_csv('C:/Users/user/Desktop/ICOdata/cagr_1year.csv', sep = ',', encoding = 'utf-8')

In [ ]:
df = pd.read_excel('C:/Users/user/Desktop/ICOdata/coingecko_return_month.xlsx', sheet_name = 'Sheet')
df = df.dropna(axis = 0, subset = ['Month_1', 'Month_12'], how = 'any')
df.reset_index(drop=True, inplace=True)
aa = []
names = []

for i in range(0, len(df)):
    name = df['Name'][i]
    cagr = 1
    for j in range(1, 13):  
        if j <= 12: 
            cagr = (1 + df[f'Month_{j}'][i]) * cagr
    a = cagr**(1/12) - 1
    aa.append(a)
    names.append(name)
# df['Name2'][10]

In [ ]:
df1 = pd.DataFrame(aa, columns = ['cagr'])
df2 = pd.DataFrame(names, columns = ['Name'])
pd.concat([df2, df1], axis = 1).to_csv('C:/Users/user/Desktop/ICOdata/coingecko_cagr_1year.csv', sep = ',', encoding = 'utf-8')

##### 對照資料

In [ ]:
inpt = pd.read_csv('C:/Users/user/Desktop/ICOdata/cagr_1year.csv', sep = ',', encoding = 'utf-8', engine = 'python')
fname = r'C:\Users\user\Desktop\ICO_face\trading_ICOname.xlsx'
file = pd.read_excel(fname, sheet_name = 'ICO_list')
aa = []
names = []
for i in range(0, len(file['ICO Name'])):
    if file['ICO Name'][i] in inpt['Name'].values:
        num = inpt[inpt.Name == file['ICO Name'][i]].index[0]
        aa.append(inpt['cagr'][num])
        names.append(file['ICO Name'][i])
    else:
        aa.append('')
        names.append(file['ICO Name'][i])

In [ ]:
dff = pd.DataFrame(aa, columns = ['cagr'])
dff2 = pd.DataFrame(names, columns = ['names'])
pd.concat([dff, dff2], axis = 1).to_csv(r'C:\Users\user\Desktop\ICOdata\cagr.csv', sep = ',', encoding = 'utf-8')

In [30]:
inpt = pd.read_csv('C:/Users/user/Desktop/ICOdata/coingecko_cagr_1year.csv', sep = ',', encoding = 'utf-8', engine = 'python')
fname = r'C:\Users\user\Desktop\ICO_face\trading_ICOname.xlsx'
file = pd.read_excel(fname, sheet_name = 'ICO_list')
aa = []
names = []
for i in range(0, len(file['ICO Name'])):
    if file['ICO Name'][i] in inpt['Name2'].values:
        num = inpt[inpt.Name2 == file['ICO Name'][i]].index[0]
        aa.append(inpt['cagr'][num])
        names.append(file['ICO Name'][i])
    else:
        aa.append('')
        names.append(file['ICO Name'][i])

In [ ]:
dff = pd.DataFrame(aa, columns = ['cagr'])
dff2 = pd.DataFrame(names, columns = ['names'])
# pd.concat([dff, dff2], axis = 1).to_csv(r'C:\Users\user\Desktop\ICOdata\gecko_cagr.csv', sep = ',', encoding = 'utf-8')
dff['cagr'].value_counts()

##### 交易量月資料

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/CMC季交易量.xlsx')

In [ ]:
# 讀取有抓到交易量的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

# path1 = 'C:/Users/user/Desktop/return_rate.xlsx'
# df_none = pd.DataFrame()
# if not os.path.isfile(path1):
#     df_none.to_excel(path1)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/CMC季交易量.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'Name'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

# 讀入CMC抓取的volume並合併
for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    s1.cell(i + 2, 1).value = dirname[i]
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除volume為0的日子
    df = df[df['volume'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    month_data = []
    month_mean = []
    # month_rate = []
    # 將資料每個月分成一組
    for j in range(0, len(df['volume']), 91):
        month_data.append(df['volume'][j:j+90])
    # 每組(每個月)volume加總算平均
    for k in range(0, len(month_data)):
        if len(month_data[k]) == 90:
            month_mean.append(month_data[k].sum()/90)
        else:
            month_mean.append(month_data[k].sum() / len(month_data[len(month_data)-1]))
        s1.cell(i + 2, k + 2).value = month_mean[k]
    # # 計算volume月成長率
    # for m in range(0, len(month_mean)-1):
    #     month_rate.append((month_mean[m+1] - month_mean[m]) / month_mean[m])
    #     s1.cell(i + 2, m + 2).value = month_rate[m]

    wb.save('C:/Users/user/Desktop/CMC季交易量.xlsx')
    # print(i)

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/gecko季交易量.xlsx')

In [ ]:
cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko2'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook(r'C:\Users\user\Desktop\gecko季交易量.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'names'
for n in range(1, 65):
    s1.cell(1, n + 1).value = f'Season_{n}'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '').lower()
    df3 = df3[df3['volum'] != 0]
    df3.reset_index(drop = True, inplace = True)
    s1.cell(i + 2, 1).value = name
    # 算市值成長率
    month_data = []
    month_mean = []
    # month_rate = []
    # 將資料每個月分成一組
    for j in range(0, len(df3['volum']), 91):
        month_data.append(df3['volum'][j:j+90])
    # 每組(每個月)volume加總算平均
    for k in range(0, len(month_data)):
        if len(month_data[k]) == 30:
            month_mean.append(month_data[k].sum()/90)
        else:
            month_mean.append(month_data[k].sum() / len(month_data[len(month_data)-1]))
    # # 計算市值月成長率
    # for m in range(0, len(month_mean)-1):
    #     month_rate.append((month_mean[m+1] - month_mean[m]) / month_mean[m])
        s1.cell(i + 2, k + 2).value = month_mean[k]
    wb.save(r'C:\Users\user\Desktop\gecko季交易量.xlsx')

##### return 3、6、12、24 month

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/ICOdata/return_361224.xlsx')

# 讀入月平均市值
path_ = 'C:/Users/user/Desktop/ICOdata/month_cap.xlsx'
df = pd.read_excel(path_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/ICOdata/return_361224.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'name'
s1['B1'].value = 'return90'
s1['C1'].value = 'return180'
s1['D1'].value = 'return365'
s1['E1'].value = 'return730'

for i in range(0, len(df)):
    # 算市值成長率
    if np.isnan(df.iloc[i, :]['Month_24']):
        s1.cell(i + 2, 1).value = df['Name'][i].lower()
        s1.cell(i + 2, 2).value = (df.iloc[i, :]['Month_3'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 3).value = (df.iloc[i, :]['Month_6'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 4).value = (df.iloc[i, :]['Month_12'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        wb.save('C:/Users/user/Desktop/ICOdata/return_361224.xlsx')
        # continue
    else:
        s1.cell(i + 2, 1).value = df['Name'][i].lower()
        s1.cell(i + 2, 2).value = (df.iloc[i, :]['Month_3'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 3).value = (df.iloc[i, :]['Month_6'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 4).value = (df.iloc[i, :]['Month_12'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 5).value = (df.iloc[i, :]['Month_24'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        wb.save('C:/Users/user/Desktop/ICOdata/return_361224.xlsx')        

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save('C:/Users/user/Desktop/ICOdata/gecko_return_361224.xlsx')

# 讀入月平均市值
path_ = 'C:/Users/user/Desktop/ICOdata/coingecko_month_cap.xlsx'
df = pd.read_excel(path_)

wb = openpyxl.load_workbook('C:/Users/user/Desktop/ICOdata/gecko_return_361224.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'name'
s1['B1'].value = 'return90'
s1['C1'].value = 'return180'
s1['D1'].value = 'return365'
s1['E1'].value = 'return730'

for i in range(0, len(df)):
    # 算市值成長率
    if np.isnan(df.iloc[i, :]['Month_12']):
        s1.cell(i + 2, 1).value = df['Name'][i].lower()
        s1.cell(i + 2, 2).value = (df.iloc[i, :]['Month_3'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 3).value = (df.iloc[i, :]['Month_6'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        wb.save('C:/Users/user/Desktop/ICOdata/gecko_return_361224.xlsx')
        # continue
    elif np.isnan(df.iloc[i, :]['Month_24']):
        s1.cell(i + 2, 1).value = df['Name'][i].lower()
        s1.cell(i + 2, 2).value = (df.iloc[i, :]['Month_3'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 3).value = (df.iloc[i, :]['Month_6'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 4).value = (df.iloc[i, :]['Month_12'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        wb.save('C:/Users/user/Desktop/ICOdata/gecko_return_361224.xlsx')
    else:
        s1.cell(i + 2, 1).value = df['Name'][i].lower()
        s1.cell(i + 2, 2).value = (df.iloc[i, :]['Month_3'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 3).value = (df.iloc[i, :]['Month_6'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 4).value = (df.iloc[i, :]['Month_12'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        s1.cell(i + 2, 5).value = (df.iloc[i, :]['Month_24'] - df.iloc[i, :]['Month_1']) / df.iloc[i, :]['Month_1']
        wb.save('C:/Users/user/Desktop/ICOdata/gecko_return_361224.xlsx')        

##### 對照資料

In [ ]:
inpt = pd.read_excel('C:/Users/user/Desktop/ICOdata/coingecko_volume_month.xlsx', sheet_name = 'Sheet2')
fname = r'C:\Users\user\Desktop\ICO_face\trading_ICOname.xlsx'
file = pd.read_excel(fname, sheet_name = 'ICO_list')
aa = []
bb = []
cc = []
dd = []
names = []
for i in range(0, len(file['names'])):
    if file['names'][i] in inpt['name'].values:
        num = inpt[inpt.name == file['names'][i]].index[0]
        aa.append(inpt['volume3m'][num])
        bb.append(inpt['volume6m'][num])
        cc.append(inpt['volume12m'][num])
        dd.append(inpt['volume24m'][num])
        names.append(file['names'][i])
    else:
        aa.append('')
        bb.append('')
        cc.append('')
        dd.append('')
        names.append(file['names'][i])

In [ ]:
dff = pd.DataFrame(names, columns = ['names'])
dff2 = pd.DataFrame(aa, columns = ['volume3m'])
dff3 = pd.DataFrame(bb, columns = ['volume6m'])
dff4 = pd.DataFrame(cc, columns = ['volume12m'])
dff5 = pd.DataFrame(dd, columns = ['volume24m'])
pd.concat([dff, dff2, dff3, dff4, dff5], axis = 1).to_csv(r'C:\Users\user\Desktop\ICOdata\gecko_volume3_24month.csv', sep = ',', encoding = 'utf-8')

In [ ]:
df = pd.read_csv(r'C:\Users\user\Desktop\ICOdata\gecko_return3_24month.csv', sep = ',', encoding = 'utf-8')
df['return90']

##### 用價格分析

In [ ]:
# workbook = openpyxl.Workbook()
# sheet = workbook.worksheets[0]
# workbook.save('C:/Users/user/Desktop/prices.xlsx')

# 讀取有抓到price的coin
dirpath_ = r'C:\Users\user\Desktop\研究所\論文\CMC歷史資料'
dirname = os.listdir(dirpath_)

# path1 = 'C:/Users/user/Desktop/return_rate.xlsx'
# df_none = pd.DataFrame()
# if not os.path.isfile(path1):
#     df_none.to_excel(path1)

# 讀入CMC抓取的市值並合併
wb = openpyxl.load_workbook('C:/Users/user/Desktop/prices.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'name'
s1['B1'].value = 'return1m'
s1['C1'].value = 'return3m'
s1['D1'].value = 'return6m'
s1['E1'].value = 'return12m'

for i in range(0, len(dirname)):
    path_ = f'C:/Users/user/Desktop/研究所/論文/CMC歷史資料/' + dirname[i]
    fnames = glob.glob(path_ + r'\*.csv', recursive = False)
    df_list = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]
    df = pd.concat(df_list, axis = 0, verify_integrity = True, ignore_index = True)
    # 去除市值為0的日子
    df = df[df['high'] != 0]
    # df = df[df['low'] != 0]
    # 重設index
    df.reset_index(drop = True, inplace = True)
    # 算市值成長率
    if len(df) < 30:
        s1.cell(i + 2, 1).value = dirname[i]
        wb.save('C:/Users/user/Desktop/prices.xlsx')
        continue
    elif 30 <= len(df) < 90:
        price_0 = df.iloc[0, :]['high']
        price_30 = df.iloc[30, :]['high']
        price_return30 = (price_30 - price_0)/ price_0
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = price_return30
        wb.save('C:/Users/user/Desktop/prices.xlsx')
    elif 90 <= len(df) < 180:
        price_0 = df.iloc[0, :]['high']
        price_30 = df.iloc[30, :]['high']
        price_90 = df.iloc[90, :]['high']
        price_return30 = (price_30 - price_0)/ price_0
        price_return90 = (price_90 - price_0)/ price_0
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = price_return30
        s1.cell(i + 2, 3).value = price_return90
        wb.save('C:/Users/user/Desktop/prices.xlsx')
    elif 180 <= len(df) < 365:
        price_0 = df.iloc[0, :]['high']
        price_30 = df.iloc[30, :]['high']
        price_90 = df.iloc[90, :]['high']
        price_180 = df.iloc[180, :]['high']
        price_return30 = (price_30 - price_0)/ price_0
        price_return90 = (price_90 - price_0)/ price_0
        price_return180 = (price_180 - price_0)/ price_0
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = price_return30
        s1.cell(i + 2, 3).value = price_return90
        s1.cell(i + 2, 4).value = price_return180
        wb.save('C:/Users/user/Desktop/prices.xlsx')
    else:
        price_0 = df.iloc[0, :]['high']
        price_30 = df.iloc[30, :]['high']
        price_90 = df.iloc[90, :]['high']
        price_180 = df.iloc[180, :]['high']
        price_365 = df.iloc[365, :]['high']
        price_return30 = (price_30 - price_0)/ price_0
        price_return90 = (price_90 - price_0)/ price_0
        price_return180 = (price_180 - price_0)/ price_0
        price_return365 = (price_365 - price_0)/ price_0
        # data = [dirname[i], return_rate180, return_rate365, return_rate730]
        # s1.append(data)
        s1.cell(i + 2, 1).value = dirname[i]
        s1.cell(i + 2, 2).value = price_return30
        s1.cell(i + 2, 3).value = price_return90
        s1.cell(i + 2, 4).value = price_return180
        s1.cell(i + 2, 5).value = price_return365
        wb.save('C:/Users/user/Desktop/prices.xlsx')

In [ ]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
workbook.save(r'C:\Users\user\Desktop\gecko_price.xlsx')

cc = r'C:\Users\user\Desktop\研究所\論文\CoinGecko3'
fnames = glob.glob(cc + r'\*.csv', recursive = False)
df2 = [pd.read_csv(fname, sep = ',', encoding = 'utf-8', engine = 'python') for fname in fnames]

wb = openpyxl.load_workbook(r'C:\Users\user\Desktop\gecko_price.xlsx', data_only = True)
s1 = wb['Sheet']
s1['A1'].value = 'name'
s1['B1'].value = 'return1m'
s1['C1'].value = 'return3m'
s1['D1'].value = 'return6m'
s1['E1'].value = 'return12m'

for i in range(0, len(fnames)):
    df3 = df2[i]
    name = fnames[i].split('\\')[-1].replace('.csv', '')
    df3 = df3[df3['prices'] != 0]
    df3.reset_index(drop = True, inplace = True)
    # 算市值成長率
    if len(df3) < 30:
        s1.cell(i + 2, 1).value = name
        wb.save(r'C:\Users\user\Desktop\gecko_price.xlsx')
        continue
    elif 30 <= len(df3) < 90:
        prices_0 = df3.iloc[0, :]['prices']
        prices_30 = df3.iloc[30, :]['prices']
        prices_return30 = (prices_30 - prices_0)/ prices_0
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = prices_return30
        wb.save(r'C:\Users\user\Desktop\gecko_price.xlsx')
    elif 90 <= len(df3) < 180:
        prices_0 = df3.iloc[0, :]['prices']
        prices_30 = df3.iloc[30, :]['prices']
        prices_90 = df3.iloc[90, :]['prices']
        prices_return30 = (prices_30 - prices_0)/ prices_0
        prices_return90 = (prices_90 - prices_0)/ prices_0
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = prices_return30
        s1.cell(i + 2, 3).value = prices_return90
        wb.save(r'C:\Users\user\Desktop\gecko_price.xlsx')
    elif 180 <= len(df3) < 365:
        prices_0 = df3.iloc[0, :]['prices']
        prices_30 = df3.iloc[30, :]['prices']
        prices_90 = df3.iloc[90, :]['prices']
        prices_180 = df3.iloc[180, :]['prices']
        prices_return30 = (prices_30 - prices_0)/ prices_0
        prices_return90 = (prices_90 - prices_0)/ prices_0
        prices_return180 = (prices_180 - prices_0)/ prices_0
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = prices_return30
        s1.cell(i + 2, 3).value = prices_return90
        s1.cell(i + 2, 4).value = prices_return180
        wb.save(r'C:\Users\user\Desktop\gecko_price.xlsx')
    else:
        prices_0 = df3.iloc[0, :]['prices']
        prices_30 = df3.iloc[30, :]['prices']
        prices_90 = df3.iloc[90, :]['prices']
        prices_180 = df3.iloc[180, :]['prices']
        prices_365 = df3.iloc[365, :]['prices']
        prices_return30 = (prices_30 - prices_0)/ prices_0
        prices_return90 = (prices_90 - prices_0)/ prices_0
        prices_return180 = (prices_180 - prices_0)/ prices_0
        prices_return365 = (prices_365 - prices_0)/ prices_0
        # data = [dirname[i], return_rate180, return_rate365, return_rate730]
        # s1.append(data)
        s1.cell(i + 2, 1).value = name
        s1.cell(i + 2, 2).value = prices_return30
        s1.cell(i + 2, 3).value = prices_return90
        s1.cell(i + 2, 4).value = prices_return180
        s1.cell(i + 2, 5).value = prices_return365
        wb.save(r'C:\Users\user\Desktop\gecko_price.xlsx')

##### 對照資料

In [338]:
file = pd.read_excel(r'C:\Users\user\Desktop\ICO_face\trading_ICOname2.xlsx', sheet_name = 'logistic')
inpt = pd.read_excel(r'C:\Users\user\Desktop\market_return.xlsx', sheet_name = 'Sheet')
aa = []
bb = []
# cc = []
# dd = []
# ee = []
# ff = []
# gg = []
# hh = []

names = []
for i in range(0, len(file['names'])):
    if file['names'][i] in inpt['names'].values:
        num = inpt[inpt.names == file['names'][i]].index[0]
        aa.append(inpt['18m_return'][num])
        bb.append(inpt['2y_return'][num])
        # cc.append(inpt['Season_3'][num])
        # dd.append(inpt['Season_4'][num])
        # ee.append(inpt['Season_5'][num])
        # ff.append(inpt['Season_6'][num])
        # gg.append(inpt['Season_7'][num])
        # hh.append(inpt['Season_8'][num])
        names.append(file['names'][i])
    else:
        aa.append('')
        bb.append('')
        # cc.append('')
        # dd.append('')
        # ee.append('')
        # ff.append('')
        # gg.append('')
        # hh.append('')
        names.append(file['names'][i])

dff = pd.DataFrame(names, columns = ['names'])
dff1 = pd.DataFrame(aa, columns = ['18m_market'])
dff2 = pd.DataFrame(bb, columns = ['2y_market'])
# dff3 = pd.DataFrame(cc, columns = ['btc_s3_mu'])
# dff4 = pd.DataFrame(dd, columns = ['btc_s4_mu'])
# dff5 = pd.DataFrame(ee, columns = ['btc_s5_mu'])
# dff6 = pd.DataFrame(ff, columns = ['btc_s6_mu'])
# dff7 = pd.DataFrame(gg, columns = ['btc_s7_mu'])
# dff8 = pd.DataFrame(hh, columns = ['btc_s8_mu'])
# , dff, dff2, dff3, dff4, dff5, dff6, dff7, dff8
pd.concat([dff, dff1, dff2], axis = 1).to_csv(r'C:\Users\user\Desktop\2y_market.csv', sep = ',', encoding = 'utf-8')